In [1]:
import pandas as pd
import pickle

In [2]:
%%time

individual_provider_rows =[]
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                         dtype={'Zip Code of the Provider': object},
                         chunksize = 50000):
    individual_provider_rows.append(chunk[chunk['Entity Type of the Provider'] == 'I']) 
               
                
individual_provider_payment_df = pd.concat(individual_provider_rows, ignore_index=True)

Wall time: 2min 37s


In [10]:
# creating total medicare revenue column

individual_provider_payment_df['Total Medicare Revenue'] = individual_provider_payment_df['Average Medicare Payment Amount'] * individual_provider_payment_df['Number of Services']

In [7]:
# creates and adds groupings for codes to dataframe

HCPCS_Grouped_Code = []

for value in individual_provider_payment_df['HCPCS Code']:
    if value.endswith("T"):
        HCPCS_Grouped_Code.append("0000T")
    else:
        HCPCS_Grouped_Code.append(value[0:3] + "00")


individual_provider_payment_df['HCPCS Grouped Code'] = HCPCS_Grouped_Code 

In [11]:
individual_provider_payment_df.shape

(9416125, 29)

In [18]:
individual_provider_payment_df.columns

Index(['National Provider Identifier',
       'Last Name/Organization Name of the Provider',
       'First Name of the Provider', 'Middle Initial of the Provider',
       'Credentials of the Provider', 'Gender of the Provider',
       'Entity Type of the Provider', 'Street Address 1 of the Provider',
       'Street Address 2 of the Provider', 'City of the Provider',
       'Zip Code of the Provider', 'State Code of the Provider',
       'Country Code of the Provider', 'Provider Type',
       'Medicare Participation Indicator', 'Place of Service', 'HCPCS Code',
       'HCPCS Description', 'HCPCS Drug Indicator', 'Number of Services',
       'Number of Medicare Beneficiaries',
       'Number of Distinct Medicare Beneficiary/Per Day Services',
       'Average Medicare Allowed Amount', 'Average Submitted Charge Amount',
       'Average Medicare Payment Amount',
       'Average Medicare Standardized Amount', 'HCPCS Grouped Code',
       'Total Medicare Revenue'],
      dtype='object')

In [20]:
individual_provider_payment_df_clean = individual_provider_payment_df.drop(columns = ['Last Name/Organization Name of the Provider',
                                                                                      'First Name of the Provider', 
                                                                                      'Middle Initial of the Provider',
                                                                                      'Entity Type of the Provider',
                                                                                      'Street Address 1 of the Provider',
                                                                                      'Street Address 2 of the Provider',
                                                                                      'City of the Provider',
                                                                                      'Zip Code of the Provider',
                                                                                      'State Code of the Provider',
                                                                                      'Country Code of the Provider',
                                                                                      'Medicare Participation Indicator',
                                                                                      'HCPCS Drug Indicator',
                                                                                      'Average Medicare Allowed Amount', 
                                                                                      'Average Submitted Charge Amount',
                                                                                      'Average Medicare Standardized Amount', 
                                                                                      ])
                                                                           
#  Keeping 'National Provider Identifier', 
#          'Credentials of the Provider',
#          'Gender of the Provider', 
#         'Provider Type', 
#         'Place of Service', 
#         'HCPCS Code',
#         'HCPCS Description', 
#         'HCPCS Grouped Code',
#         'Total Medicare Revenue', 
#         'Average Medicare Payment Amount',
#         'Number of Services',
#         'Number of Medicare Beneficiaries',
#         'Number of Distinct Medicare Beneficiary/Per Day Services'


In [25]:
individual_provider_payment_df_clean.describe()

,National Provider Identifier,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Payment Amount,Total Medicare Revenue
count,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06
mean,1.499785e+09,1.983183e+02,7.084164e+01,1.151297e+02,7.392625e+01,7.316105e+03
std,2.877289e+08,2.351712e+03,1.442648e+02,2.783331e+02,1.705221e+02,2.952656e+04
min,1.003000e+09,2.400000e+00,1.100000e+01,1.100000e+01,0.000000e+00,0.000000e+00
25%,1.255302e+09,2.000000e+01,1.700000e+01,2.000000e+01,1.967050e+01,7.561600e+02
50%,1.497969e+09,4.300000e+01,3.200000e+01,4.000000e+01,4.745194e+01,2.079750e+03
75%,1.740681e+09,1.160000e+02,7.300000e+01,1.040000e+02,8.456263e+01,6.012600e+03
max,1.993000e+09,3.301134e+06,9.532700e+04,1.758700e+05,3.125351e+04,9.417461e+06


In [26]:
individual_provider_payment_df_clean.head()

,National Provider Identifier,Credentials of the Provider,Gender of the Provider,Provider Type,Place of Service,HCPCS Code,HCPCS Description,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Payment Amount,HCPCS Grouped Code,Total Medicare Revenue
0,1003000126,M.D.,M,Internal Medicine,F,99217,Hospital observation care discharge,100.0,96,100,56.827200,99200,5682.72
1,1003000126,M.D.,M,Internal Medicine,F,99218,Hospital observation care typically 30 minutes,26.0,25,26,78.460000,99200,2039.96
2,1003000126,M.D.,M,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,52.0,51,52,102.807692,99200,5346.00
3,1003000126,M.D.,M,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,59.0,59,59,141.293559,99200,8336.32
4,1003000126,M.D.,M,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",16.0,16,16,79.710000,99200,1275.36


In [24]:
individual_provider_payment_df_clean.to_csv("../data/final_medicare_provider_file.csv", index = False)


In [7]:
# optional pickle

# individual_provider_payment_df_clean.to_pickle("../data/individual_provider_payment.pkl")